# Aplicação de Python na Agrometeorologia

Este Jupyter Notebook é parte integrande do curso de Python ministrado durante XXII Congresso Brasiliero de Agrometeorologia realizado nos dias 3, 4 e 5 de outubro de 2023 na cidade Natal/RN.

# Importação de bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Abertura do arquivo csv

Tratamento da coluna 'Dia' do DataFrame como DateTime e depois converte em index.

In [ ]:
# Documentação:
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

df1 = pd.read_csv(
    '../output/chuva_diaria_cpc_19910901_20221231.csv',
    index_col='Dia', 
    parse_dates=True  
)

In [ ]:
#Outra possibilidade de tratar a coluna 'Dia' como DateTime e depois converter em index:
# df1['Dia'] = pd.to_datetime(df1['Dia'])
# df1.set_index('Dia', inplace=True)

Inspeção do DataFrame.

In [ ]:
df1

Plot básico do DataFrame.

In [ ]:
df1['Chuva'].plot() # Outra possibilidade: df.Chuva.plot()

# Algumas transformações no DataFrame

## Anomalia mensal

Anomalia = Valor observado - Climatologia

In [ ]:
# Acumulado mensal.
chuva_acumulada = df1.resample('MS').sum()

# Formata a data de forma a remover o dia.
chuva_acumulada.index = chuva_acumulada.index.strftime('%Y-%m')

# Renomeia o index de 'Dia' para 'Mes'.
chuva_acumulada.index.name = 'Mes'

chuva_acumulada

In [ ]:
# Uso do reshape para organizar o dado no formato vetor para o formato tabular (linhas x colunas).
# 32 = anos.
# 12 = meses.

dado_reshape = chuva_acumulada['Chuva'].values.reshape(32,12)

# Nome dos rótulos das colunas.
colunas = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Nome dos rótulos do index.
index = range(1991, 2023)

# Cria o DataFrame.
df2 = pd.DataFrame(
    data=dado_reshape, 
    index=index, 
    columns=colunas
)

df2

In [ ]:
df2.loc[1991]

In [ ]:
# axis = 0 -> faz cálculo nas linhas.
# axis = 1 -> faz cálculo nas colunas.

# Seleciona os anos de interesse.
AnoInicial, AnoFinal = 1991, 2020

climatologia = df2.loc[AnoInicial:AnoFinal].mean(axis=0)

climatologia

In [ ]:
lista = []
anos = []

for ano in range(1991, 1993):
    anomalia = df2.loc[ano] - climatologia
    lista.append(anomalia)
    anos.append(ano)

In [ ]:
df3 = pd.DataFrame(lista, index=anos)

df3

O stack faz o empilhamento dos anos.

In [ ]:
df4 = df3.stack().reset_index()

df4

Renomeia as colunas.

In [ ]:
df4.rename(
    columns={'level_0':'Ano', 
             'level_1':'Mes', 
             0:'Chuva'},
    inplace=True
)

In [ ]:
df4

In [ ]:
# Dicionário de meses para converte de mês string para mês inteiro (1, 2, ..., 12).

meses = {
    'Jan': 1, 
    'Fev': 2, 
    'Mar': 3, 
    'Abr': 4, 
    'Mai': 5, 
    'Jun': 6, 
    'Jul': 7, 
    'Ago': 8, 
    'Set': 9, 
    'Out': 10, 
    'Nov': 11,
    'Dez': 12
}

Substitui mês string por mes numérico inteiro.

In [ ]:
df4['Mes'] = df4['Mes'].replace(meses)

In [ ]:
df4

Cria uma coluna chamada 'Data' no DataFrame. Junta as colunas 'Ano' e 'Mes' para gerar a coluna com a data formatada.

In [ ]:
df4['Data'] = pd.to_datetime(
    df4[['Ano', 'Mes']].astype(str).agg(' '.join, axis=1), 
    format='%Y %m'
)

In [ ]:
df4

In [ ]:
# Remove colunas desnecessárias.
df4.drop(
    columns=['Ano', 'Mes'], 
    inplace=True
)

# Define a coluna 'Data' como index.
df4.set_index(
    'Data', 
    inplace=True
)

# Formata a data de forma a remover o dia.
df4.index = df4.index.strftime('%Y-%m')

df4

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

df4['Chuva'].plot.bar()